# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
# 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [2]:
# 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
!tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [18]:
# 필요한 라이브러리를 설치합니다.
!pip install timm

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [34]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [35]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [21]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [22]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'efficientnet_b3' # 'efficientnet_b1' #'resnet50' 'resnet34' # , ...

# training config - 이 부분을 수정하여 성능을 높일 수 있음 
img_size = 384
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 32
num_workers = 8 # 속도 향상을 위해 조정
N_SPLITS = 5  # 5-Fold 교차 검증
EARLY_STOPPING_PATIENCE = 5 # 5 에포크 동안 점수 향상 없으면 중단


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [23]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),

    # 극심한 회전 및 원근 왜곡 추가
    A.Perspective(scale=(0.05, 0.1), pad_mode=0, p=0.7),
    A.Affine(
        rotate=(-90, 90),       # -90도 ~ +90도 사이로 마구 회전
        shear=(-15, 15),       # 이미지 기울임
        scale=(0.8, 1.2),      # 80%~120% 확대/축소
        translate_percent=0.1, # 상하좌우 이동
        p=0.8
    ),
    # 조명 및 품질 저하 흉내내기
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.8),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=0.5),
        A.MotionBlur(blur_limit=(3, 7), p=0.5),
    ], p=0.5),
    # 이미지 일부 가리기 (과적합 방지)
    A.CoarseDropout(max_holes=8, max_height=img_size//8, max_width=img_size//8, min_holes=1, p=0.7),
    # 좌우 반전
    # 자동차 사진도 있고, 문서도 뒤집혀 있으므로 HorizontalFlip은 좋은 증강입니다.
    A.HorizontalFlip(p=0.5),

    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [ ]:
# Validation 함수
def validate_one_epoch(loader, model, loss_fn, device):
    model.eval() # 모델을 평가 모드로 설정
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad(): # 기울기 계산 비활성화
        pbar = tqdm(loader, desc="Valid")
        for image, targets in pbar:
            image = image.to(device)
            targets = targets.to(device)
            
            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
            pbar.set_description(f"Valid Loss: {loss.item():.4f}")

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro') # Macro F1

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }
    return ret

In [25]:
# [수정] K-Fold 호환되는 ImageDataset 클래스
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None, target_col='target'):
        self.df = df 
        self.path = path
        self.transform = transform
        self.target_col = target_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name = self.df.iloc[idx]['ID']
        target = self.df.iloc[idx][self.target_col]
        
        # 이미지 열기
        img_path = os.path.join(self.path, name)
        try:
            img = np.array(Image.open(img_path))
        except FileNotFoundError:
            print(f"Error: Image file not found at {img_path}")
            # 또는 기본 이미지 반환
            return torch.randn(3, img_size, img_size), 0 
            
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 전체 train.csv를 먼저 불러옵니다.
print("Loading main train.csv...")
df = pd.read_csv("datasets_fin/train.csv")

# StratifiedKFold를 정의합니다.
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# 'target' (레이블)을 기준으로 데이터를 N_SPLITS개로 나눕니다.
# fold 컬럼이 0, 1, 2, 3, 4 값으로 채워집니다.
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['target'])):
    df.loc[val_idx, 'fold'] = fold

print("Stratified K-Fold split complete.")

all_folds_best_f1 = []
# 각 Fold를 순회하며 학습 및 검증
for fold in range(N_SPLITS):
    print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    
    # fold 컬럼을 기준으로 train / validation DataFrame을 나눔
    train_df = df[df['fold'] != fold].reset_index(drop=True)
    val_df = df[df['fold'] == fold].reset_index(drop=True)
    
    # Dataset 및 DataLoader 정의
    trn_dataset = ImageDataset(
        train_df,
        "datasets_fin/train/",
        transform=trn_transform,
        target_col='target'
    )
    val_dataset = ImageDataset(
        val_df, 
        "datasets_fin/train/", 
        transform=val_transform,
        target_col='target'
    )

    # DataLoader 정의
    trn_loader = DataLoader(
        trn_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    # load model
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=17
    ).to(device)
    
    # Loss 가중치 계산
    current_num_classes = 17
    weights = torch.ones(current_num_classes, dtype=torch.float32).to(device)
    
    class_counts = train_df['target'].value_counts()
    
    if not class_counts.empty:
        weight_values = 1. / torch.tensor(class_counts.values, dtype=torch.float32).to(device)
        weights.scatter_(0, torch.tensor(class_counts.index, dtype=torch.long).to(device), weight_values)

    weights = (weights / weights.sum()) * current_num_classes # (17.0)
    loss_fn = nn.CrossEntropyLoss(weight=weights)

    # Weight Decay 추가
    optimizer = Adam(model.parameters(), lr=LR, weight_decay=0)

    # [참고] LR 스케줄러 (예: CosineAnnealingLR)를 추가하면 성능 향상에 도움됩니다.
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

    # Early Stopping을 위한 변수
    best_val_f1 = 0.0
    patience_counter = 0

    for epoch in range(EPOCHS):
        start_time = time.time()

        train_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
        val_ret = validate_one_epoch(val_loader, model, loss_fn, device)
        train_ret['epoch'] = epoch

        elapsed = time.time() - start_time

        # 스케줄러 추가
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{EPOCHS} [{elapsed:.0f}s] - "
              f"Train F1: {train_ret['train_f1']:.4f}, "
              f"Valid F1: {val_ret['val_f1']:.4f}, " # 이 점수가 제일 중요
              f"Valid Loss: {val_ret['val_loss']:.4f}")

        # Early Stopping 및 모델 저장 로직
        if val_ret['val_f1'] > best_val_f1:
            best_val_f1 = val_ret['val_f1']
            # 가장 점수가 좋은 모델을 저장합니다 (fold 번호 포함)
            torch.save(model.state_dict(), f"best_model_fold_{fold}.pth")
            print(f"⭐️ Best F1 Score updated to {best_val_f1:.4f}. Model saved!")
            patience_counter = 0 # 초기화
        else:
            patience_counter += 1
        
        if patience_counter >= EARLY_STOPPING_PATIENCE:
            print(f"Early stopping at epoch {epoch+1} as F1 score did not improve for {EARLY_STOPPING_PATIENCE} epochs.")
            break

    # 최고 점수(best_val_f1)를 리스트에 저장
    print(f"Fold {fold+1} Best F1 Score: {best_val_f1:.4f}")
    all_folds_best_f1.append(best_val_f1)
            
print("--- All Folds Training Complete ---")

final_cv_score = np.mean(all_folds_best_f1)

print(f"\n--- 5-Fold Cross-Validation Summary ---")
print(f"Individual Fold F1 Scores: {all_folds_best_f1}")
print(f"Average CV F1 Score (Final Score): {final_cv_score:.4f}")

Stratified K-Fold split complete.
--- Fold 1/5 ---


Valid Loss: 0.3252: 100%|██████████| 10/10 [00:01<00:00,  8.98it/s]


Epoch 1/100 [11s] - Train F1: 0.7973, Valid F1: 0.8506, Valid Loss: 5.5520
⭐️ Best F1 Score updated to 0.8506. Model saved!


Valid Loss: 0.0318: 100%|██████████| 10/10 [00:01<00:00,  8.87it/s]


Epoch 2/100 [11s] - Train F1: 0.9198, Valid F1: 0.9300, Valid Loss: 0.1913
⭐️ Best F1 Score updated to 0.9300. Model saved!


...

Epoch 7/100 [11s] - Train F1: 0.9597, Valid F1: 0.9874, Valid Loss: 0.0346


Valid Loss: 0.0013: 100%|██████████| 10/10 [00:01<00:00,  8.44it/s]


Epoch 8/100 [11s] - Train F1: 0.9265, Valid F1: 0.9814, Valid Loss: 0.0542


Valid Loss: 0.0010: 100%|██████████| 10/10 [00:01<00:00,  8.32it/s]

Epoch 9/100 [11s] - Train F1: 0.9610, Valid F1: 0.9812, Valid Loss: 0.0295
Early stopping at epoch 9 as F1 score did not improve for 5 epochs.
Fold 5 Best F1 Score: 0.9877
--- All Folds Training Complete ---

--- 5-Fold Cross-Validation Summary ---
Individual Fold F1 Scores: [0.9935846560846561, 0.9937241200828157, 0.9968452206719354, 0.9815496131977763, 0.9876904671137733]
Average CV F1 Score (Final Score): 0.9907


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [ ]:
tst_df = pd.read_csv("datasets_fin/sample_submission.csv")
tst_dataset = ImageDataset(
    tst_df,
    "datasets_fin/test/",
    transform=tst_transform  # Test 셋에는 증강이 없는 val_transform 사용
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE * 2, # BATCH_SIZE * 2 로 키워도 좋습니다 (추론 속도 향상)
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# 2. 5개 모델의 예측 확률을 저장할 리스트 준비
# (3140, 17) 크기의 0으로 채워진 배열을 5개 만듦
all_fold_preds = []

# 3. 5개 Fold 루프를 돌며 추론
for fold in range(N_SPLITS):
    print(f"--- Inference for Fold {fold+1}/{N_SPLITS} ---")
    
    # 3-1. 모델을 *새로* 초기화하고 저장된 가중치(weight)를 불러옴
    model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device) # pretrained=False
    model.load_state_dict(torch.load(f"best_model_fold_{fold}.pth"))
    model.eval()
    
    # TTA 적용
    fold_preds_list = []
    with torch.no_grad():
        for image, _ in tqdm(tst_loader):
            image = image.to(device)
            
            pred_orig = model(image)

            # 2. TTA: 좌우 반전(Horizontal Flip)된 이미지 예측
            # torch.flip으로 텐서를 직접 뒤집음 (dims=[-1]이 좌우반전)
            image_hflip = torch.flip(image, dims=[-1]) 
            pred_hflip = model(image_hflip)
            
            # (선택적 TTA) 90도 회전 예측
            # image_rot90 = torch.rot90(image, k=1, dims=[-2, -1]) # 90도
            # pred_rot90 = model(image_rot90)

            # 3. [중요] 예측 *확률*(softmax)들을 평균냄
            # (원본 50%, 좌우반전 50% 가중치)
            avg_batch_pred = (pred_orig.softmax(dim=1) + pred_hflip.softmax(dim=1)) / 2.0
            
            # (만약 90도 회전도 했다면 / 3.0 으로 나눠야 함)
            # avg_batch_pred = (pred_orig.softmax(dim=1) + pred_hflip.softmax(dim=1) + pred_rot90.softmax(dim=1)) / 3.0

            # 평균낸 확률을 리스트에 저장
            fold_preds_list.append(avg_batch_pred.cpu().numpy())
            
            
    # 3-2. 이번 Fold의 예측값을 하나로 합침 (3140, 17)
    fold_preds_array = np.concatenate(fold_preds_list, axis=0)
    all_fold_preds.append(fold_preds_array)

# 4. 5개 Fold의 예측 확률을 평균냄 (Ensemble)
# all_fold_preds 리스트에는 (3140, 17) 배열이 5개 들어있습니다.
# (5, 3140, 17) -> (3140, 17)
avg_preds = np.mean(all_fold_preds, axis=0)

# 5. [중요] 평균낸 확률값에서 최종 클래스(argmax)를 구함
final_preds = np.argmax(avg_preds, axis=1)

# 6. 제출 파일 생성
pred_df = tst_df.copy()
pred_df['target'] = final_preds

In [32]:
assert (tst_df['ID'] == pred_df['ID']).all()

In [ ]:
pred_df.to_csv("pred/pred10.csv", index=False)
print("Submission file created!")

Submission file created!
